In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [29]:
import os
import glob

KeyboardInterrupt: 

In [30]:
train_path = r'imgs_2023050915314323740/train_test_dataset_Fault Impact Analysis'
train_files = glob.glob(os.path.join(train_path, "*.csv"))

def one_hot_encoder(hour):
    if 0 <= hour < 24:
        one_hot_vector = [0] * 24
        one_hot_vector[hour] = 1
        return one_hot_vector
    else:
        raise ValueError("Hour must be in the range 0 to 23.")

In [5]:
mean = np.array([0,0,0,0,0,0,0,0,0])
std = np.array([0,0,0,0,0,0,0,0,0])
count = 0

for file in train_files:
    df = pd.read_csv(file)
    df = df.fillna(0)
    columns = df.columns
    mean_1 = df[columns[2:]].mean()
    mean = mean + mean_1

    std_1 = df[columns[2:]].std()
    std = std + std_1
    count += 1

mean = mean/count
std = std/count

In [36]:
test_file = pd.read_csv(train_files[0])
cols = test_file.columns
diff = test_file[cols[2:-2]].diff().values

x_pred = (test_file[cols[2:-2]] - mean[:-2]) / std[:-2]
x_diff = x_pred.diff()
x_diff.iloc[0] = [0]*7

x_diff

,access_success_rate,resource_utilition_rate,TA,bler,cqi,mcs,data_rate
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.042375,-1.293460,0.330149,-0.460054,0.105545,-0.945294,0.584169
2,0.103933,1.100632,-0.094861,0.458637,0.515699,1.745488,-1.392725
3,0.005469,-0.772939,-0.063025,-0.188717,0.176865,0.379875,1.893932
4,-0.006516,-0.111527,-0.129429,0.238300,-0.188622,0.117643,-0.239398
...,...,...,...,...,...,...,...
272,0.021003,0.492209,-0.027626,1.012789,-0.061761,1.716646,-0.151530
273,0.000000,2.782427,0.021123,0.330748,-0.182530,-0.591072,-1.517395
274,0.000000,0.029269,0.228027,0.005583,-0.034120,-0.007968,-0.288785
275,0.000000,-1.533922,-0.208165,0.035091,-0.313970,-0.630791,0.150728


In [38]:

#create data sets:
X = np.empty((1, 40), dtype='float32')
Y = np.array([])
print("X::::", X)
for file in train_files:

    df = pd.read_csv(file)
    columns = df.columns
    df[columns[2:]] = df[columns[2:]].fillna(mean)  # Fill NaN values with the mean.

    # Convert endTime column to datetime.
    df['endTime'] = pd.to_datetime(df['endTime'])

    # Calculate fault_duration using vectorized operations.
    fault_duration = (df['fault_duration'] - mean[-2]) / std[-2]
    fault_duration = np.array(fault_duration)
    # Calculate x_pred using vectorized operations.
    x_pred = (df[columns[2:-2]] - mean[:-2]) / std[:-2]
    #calculate diff between t-1 and t-2
    x_diff = x_pred.diff()
    x_diff.iloc[0] = [0]*7

    x_diff = np.array(x_diff)
    x_pred = np.array(x_pred)
    # One-hot encode the hour using vectorized operations.
    hour_enc = np.array([one_hot_encoder(hour) for hour in df['endTime'].dt.hour])

    relation = np.array(df['relation'].values)
    # Calculate diff_data_rate using vectorized operations.
    diff_data_rate = df['data_rate'].diff().values

    # Generate Y array using vectorized operations.
    Y = np.append(Y, (diff_data_rate < 0).astype(int)[1:])

    # Concatenate all the features using vectorized operations.
    features = np.concatenate((x_pred[:-1], x_diff[:-1],
                               fault_duration[1:, None],
                               relation[1:, None],
                               hour_enc[:-1]),
                              axis=1)


    X = np.append(X, features, axis=0)




X = np.array(X[1:])
print(X.shape, Y.shape)

X:::: [[1.46911760e+32 6.16571324e-43 1.46911760e+32 6.16571324e-43
  1.46911760e+32 6.16571324e-43 1.46911760e+32 6.16571324e-43
  1.46911760e+32 6.16571324e-43 1.46911760e+32 6.16571324e-43
  1.46911760e+32 6.16571324e-43 1.46911760e+32 6.16571324e-43
  1.46911760e+32 6.16571324e-43 1.46911760e+32 6.16571324e-43
  6.48227959e+31 6.16571324e-43 1.48973395e+32 6.16571324e-43
  1.48977263e+32 6.16571324e-43 1.48981906e+32 6.16571324e-43
  1.48984227e+32 6.16571324e-43 1.50209942e+32 6.16571324e-43
  1.50210716e+32 6.16571324e-43 1.50211490e+32 6.16571324e-43
  1.50212263e+32 6.16571324e-43 1.50213037e+32 6.16571324e-43]]
(901666, 40) (901666,)


In [39]:
np.save(r'Features.npy', X)
np.save(r'Target.npy', Y)

In [40]:
#create fine tuning data points
X_fine = []
Y_fine = np.array([])

for file in train_files:

    df = pd.read_csv(file)
    columns = df.columns
    df[columns[2:]] = df[columns[2:]].fillna(mean)
    df['endTime'] = pd.to_datetime(df['endTime'])

    for index in range(1,len(df)):

        if df['fault_duration'].iloc[index] != 0:

            fault_duration = (df['fault_duration'].iloc[index] - mean[-2]) / std[-2]
            relation = df['relation'].iloc[index]

            x_pred = (df[columns[2:-2]].iloc[index-1] - mean[:-2]) / std[:-2]

            hour = int(df['endTime'].iloc[index-1].hour)
            hour_enc = one_hot_encoder(hour)


            diff_data_rate = df['data_rate'].iloc[index] - df['data_rate'].iloc[index-1]

            if diff_data_rate < 0:
                Y_fine = np.append(Y_fine, 1)

            else:
                Y_fine = np.append(Y_fine, 0)


            if index > 1:
                x_t2 = (df[columns[2:-2]].iloc[index-2] - mean[:-2]) / std[:-2]

                x_diff = x_pred - x_t2
            else:
                x_diff = np.array([0]*7)



            features = np.concatenate((x_pred, x_diff, [fault_duration], [relation], hour_enc),
                                      axis =0)

            X_fine.append(features)

            break

X_fine = np.array(X_fine)

print(X_fine.shape, Y_fine.shape)

(7256, 40) (7256,)


In [41]:
nan_mask = np.isnan(X_fine)
nan_count = np.sum(nan_mask)
nan_count

0

In [42]:
np.save(r'Features_fine.npy', X_fine)
np.save(r'Target_fine.npy', Y_fine)

In [43]:
val_folder_path = r'validation_clean/'
sample_submission = pd.read_csv('SampleSubmission.csv')
indexes = sample_submission['ID'].values

X_val = []

for id in indexes:

    df = pd.read_csv(val_folder_path + id +'.csv.csv')

    columns = df.columns
    df[columns[2:]] = df[columns[2:]].fillna(mean)
    df['endTime'] = pd.to_datetime(df['endTime'])

    for index in range(1,len(df)):

        if df['fault_duration'].iloc[index] != 0:

            fault_duration = (df['fault_duration'].iloc[index] - mean[-2]) / std[-2]
            relation = df['relation'].iloc[index]

            x_pred = (df[columns[3:-2]].iloc[index-1] - mean[:-2]) / std[:-2]


            hour = int(df['endTime'].iloc[index-1].hour)
            hour_enc = one_hot_encoder(hour)

            if index > 1:

                x_t2 = (df[columns[3:-2]].iloc[index-2] - mean[:-2]) / std[:-2]
                x_diff = x_pred - x_t2
            else:
                x_diff = np.array([0]*7)



            features = np.concatenate((x_pred, x_diff, [fault_duration], [relation], hour_enc),
                                      axis =0)

            X_val.append(features)

            break


X_val = np.array(X_val)
print(X_val.shape)

(1932, 40)


In [45]:
nan_mask = np.isnan(X_val)
nan_count = np.sum(nan_mask)
nan_count

0

In [46]:
np.save(r'Features_val.npy', X_val)